PARA NOVO DF

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Inicialize a sessão do Spark
spark = SparkSession.builder \
    .appName("Generate Surrogate Key") \
    .getOrCreate()

# Carregue seus dados
data = [(1, "John"),
        (2, "Jane"),
        (3, "Bob"),
        (4, "Alice")]

df = spark.createDataFrame(data, ["id", "name"])

# Defina uma janela para a função row_number()
windowSpec = Window.orderBy("id")

# Adicione uma coluna com a surrogate key
df = df.withColumn("surrogate_key", row_number().over(windowSpec))

# Mostre o resultado
df.show()

24/06/07 14:41:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/07 14:41:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/07 14:41:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+-----+-------------+
| id| name|surrogate_key|
+---+-----+-------------+
|  1| John|            1|
|  2| Jane|            2|
|  3|  Bob|            3|
|  4|Alice|            4|
+---+-----+-------------+



PARA APPEND

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Inicialize a sessão do Spark
spark = SparkSession.builder \
    .appName("Append Surrogate Key") \
    .getOrCreate()

# Carregue seus dados existentes
existing_data = [(1, "John"),
                 (2, "Jane"),
                 (3, "Bob"),
                 (4, "Alice")]

df_existing = spark.createDataFrame(existing_data, ["id", "name"])

# Carregue seus novos dados
new_data = [(5, "Eva"),
            (6, "Mike"),
            (7, "Lisa")]

df_new = spark.createDataFrame(new_data, ["id", "name"])

# Determine o valor máximo da surrogate key existente
max_existing_key = df.selectExpr("max(surrogate_key) as max_key").collect()[0]["max_key"]

# Adicione uma coluna com a surrogate key para os novos dados
windowSpec = Window.orderBy("id")
df_new = df_new.withColumn("surrogate_key", row_number().over(windowSpec) + max_existing_key)

# Append dos novos dados ao DataFrame existente
df_combined = df.union(df_new)

# Mostre o resultado
df_combined.show()

24/06/07 14:46:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/07 14:46:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/07 14:46:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/07 14:46:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/07 14:46:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/07 14:46:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/07 1

+---+-----+-------------+
| id| name|surrogate_key|
+---+-----+-------------+
|  1| John|            1|
|  2| Jane|            2|
|  3|  Bob|            3|
|  4|Alice|            4|
|  5|  Eva|            5|
|  6| Mike|            6|
|  7| Lisa|            7|
+---+-----+-------------+

